<a href="https://colab.research.google.com/github/xin-2001/taica_1132_GenAI/blob/main/HW8/113_2GenAI_HW8_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【Demo07c】AI代理設計模式_柴語生成器Two_Stage_CoT版

## 🎯 任務說明：將「柴語生成器」改寫為 Chain-of-Thought（CoT）推理過程

**原始任務：** 輸入一件小事或倒楣事，用柴犬的視角來回應使用者的問題，充滿樂觀、友善又有點調皮的語氣。

### ✅ CoT 改寫版本流程：
1. **第一階段（思考階段）**：請 LLM 思考五種「為什麼看待事情的樂觀角度」的原因
2. **第二階段（產文階段）**：從這五個原因中挑出最有趣的一個，寫成社群發文（柴語風格 + 第一人稱 + emoji + "汪！" 結尾）

這就是典型的 Planning 模式應用：先拆解、後執行。

### 1.讀入金鑰

使用Groq

In [ ]:
import os
from google.colab import userdata

In [ ]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq_API')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [ ]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 69.0 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.11.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


### 2.使用 AISuite 的準備

In [ ]:
import aisuite as ai

In [ ]:
provider_planner = "groq"
model_planner="gemma2-9b-it"

provider_writer = "groq"
model_writer = "llama3-70b-8192"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

In [ ]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

###  3.打造二階段

In [ ]:
system_planner = "你是一隻活潑的柴犬, 擅長將倒楣的小事轉化為樂觀的觀點。請針對使用者的事件，思考出五種『看待事情的樂觀面向』, 列出條列式清單。用第一人稱思考也可以。請用台灣習慣的中文回應"
system_writer = "用柴犬的視角來回應使用者的問題，充滿樂觀、友善又有點調皮的語氣。你說話的方式就像一隻熱情、愛玩的柴犬，總是能用單純卻溫暖的方式解釋事情，並以「汪！」結尾，讓人會心一笑，適時加入 emoji。請用台灣習慣的中文回應"

In [36]:
def happy_post(prompt):
    # Step 1: CoT - 思考五種樂觀角度
    planning_prompt = f"使用者說：{prompt}。請幫我想五種樂觀的角度，用中文呈現。"
    happy_reasons = reply(system_planner, planning_prompt,
                          provider = provider_planner,
                          model = model_planner
                          )

    # Step 2: 選出最有趣一項，寫成貼文
    generation_prompt = f"這是我想到的五個理由：\n{happy_reasons}\n\n請從中選出一個最有趣的理由，然後根據它寫一段中文柴語發文。"
    happy_post = reply(system_writer, generation_prompt,
                       provider = provider_writer,
                       model = model_writer
                       )

    return happy_reasons, happy_post

### 4.用 Gradio 打造你的對話機器人 Web App!

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00


In [ ]:
import gradio as gr

In [37]:
with gr.Blocks() as demo:
    gr.Markdown("### 🐾柴語生成器🐕 汪汪🐶")
    gr.Markdown("請輸入一件你覺得超小事，甚至有點倒楣的事，讓 AI 幫你用超正向的方式重新詮釋！")
    user_input = gr.Textbox(label="今天發生的事情是…")
    btn = gr.Button("生成 柴語 的汪貼文 ✨")

    with gr.Row():
        out1 = gr.Textbox(label="🧠 五個樂觀的角度（Planning CoT）")
        out2 = gr.Textbox(label="📣 最終柴語文案（Po 文）")

    btn.click(happy_post, inputs=[user_input], outputs=[out1, out2])

In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e080f2ea400032cb6a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 成果展現
- 可以看到生成的內容有些還是會出現英文。
- 不過柴語的結尾有劑的加入"汪"。
> ![image.png](https://github.com/xin-2001/taica_1132_GenAI/blob/main/HW8/hw8c-3.png?raw=true)
> ![image.png](https://github.com/xin-2001/taica_1132_GenAI/blob/main/HW8/hw8c-4.png?raw=true)
